# Data Splitting

In this notebook I'm splitting and saving the data for sentiment analysis

## Imports

In [1]:
import gc
import os
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
SEED = 42


## Paths

In [3]:
relative_path = os.path.join("../../../", "data")


In [4]:
cleaned_reviews_path = os.path.join(relative_path, "2_cleaned_reviews")


In [5]:
sentiment_analysis_data_path = os.path.join(relative_path, "3_sentiment_analysis")
Path(sentiment_analysis_data_path).mkdir(parents=True, exist_ok=True)


## Data

### Loading data

In [6]:
reviews_df = pd.read_parquet(os.path.join(cleaned_reviews_path, "reviews.parquet"))

### Dataset overview

In [7]:
reviews_df.sample(n=10, random_state=SEED)


,show_id,user_id,type,datetime,sentiment,subtitle,review,review_score,score
196236,257386,28525,series,2010-08-27 11:23:00,positive,Вечный город.,К истории Древнего мира у меня отношение особо...,40,NaN
128582,688832,44953,movie,2015-02-20 23:19:00,negative,50 оттенков разочарования,"Говорю сразу, книги читала все, да и по нескол...",5,3.0
159673,349,33910,movie,2018-04-15 21:04:00,positive,"Господи, спасибо, что не пронесло мимо","Есть два типа фильмов, мой друг. Одни ты прост...",4,NaN
109244,686898,44065,movie,2019-11-09 11:58:00,neutral,Что же стало с клоуном?,"Итак, в первую очередь хотелось бы отметить то...",2,6.5
92610,61455,66782,movie,2017-11-27 18:52:00,positive,Они отказываются подчиняться,"Автора этого замечательного фильма, Джосса Уэд...",5,NaN
43727,491724,44563,movie,2012-01-28 23:18:00,positive,Жестокая правда,"Финчер снова нас поразил, он всегда нас поража...",7,10.0
50195,102130,67145,movie,2009-08-17 11:56:00,positive,"Преодолеть 2 года жизни, что встретиться","Слышал о фильме много, и в основном положитель...",11,10.0
43651,491724,66283,movie,2012-02-19 15:33:00,negative,"Мужчины, которые ненавидели женщин.",Я попробовала рассматривать этот фильм с двух ...,3,6.0
48775,7226,7905,movie,2014-02-20 03:44:00,positive,None,"«Догвилль» - это один из тех редких фильмов, п...",3,10.0
75262,458,33255,movie,2013-06-22 21:14:00,positive,Тайна закрытой двери,Я имела счастье смотреть этот мультфильм в кин...,6,10.0


### Looking at reviews

In [8]:
for review in reviews_df.sample(n=10, random_state=SEED)["review"].values[:3]:
    print(review.replace("<p>", "\n"))
    print("\n")


К истории Древнего мира у меня отношение особое, как и к самой истории в целом. Меня манит великолепие дворцов разных эпох, сильные характеры и яркие личности, грандиозные и эпические битвы. Я обожаю историю Древнего Рима, Греции, Вавилонии, Египта, обожаю историю Средневековья. Есть в прошлом что-то такое притягательное и таинственное, то, что манит. Иногда мне кажется, что я человек той эпохи, а не этой. Поэтому, в силу таких моих личных взглядов исторические картины представляют для меня наибольший интерес. И именно поэтому совершенно неудивительно, что наткнувшись на сериал 'Рим' я им серьёзно увлеклась.
 Римская империя - одна из самых известных цивилизаций, которая положила начало многим основам нашей современной жизни. Римом правили по истине великие люди. Именно Рим дал нам понятие демократии, сената, республики и многих других политических вещей. До сих пор многие римские законы действуют в нашем обществе, конечно, видоизменённые под наш век. Но факт остаётся фактом, Римская и

In the previous step I've removed the scores from the reviews, so it is now safe to continue with baseline model creation.

### Selecting needed columns

For baseline model we're interested only in `sentiment` and `review_body` columns

In [9]:
df = reviews_df[["sentiment", "review"]]


In [10]:
del reviews_df
gc.collect()


0

### Splitting the data

In [11]:
train_df, test_df = train_test_split(
    df, test_size=0.1, random_state=SEED, stratify=df["sentiment"]
)
train_df.shape, test_df.shape


((186063, 2), (20674, 2))

In [12]:
train_df["sentiment"].value_counts(normalize=True)


positive    0.720331
neutral     0.149971
negative    0.129698
Name: sentiment, dtype: float64

In [13]:
test_df["sentiment"].value_counts(normalize=True)


positive    0.720325
neutral     0.149995
negative    0.129680
Name: sentiment, dtype: float64

#### Saving split data

In [14]:
train_df["fold"] = "train"
test_df["fold"] = "test"


In [15]:
data = pd.concat([train_df, test_df]).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206737 entries, 0 to 206736
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   sentiment  206737 non-null  category
 1   review     206737 non-null  object  
 2   fold       206737 non-null  object  
dtypes: category(1), object(2)
memory usage: 3.4+ MB


In [16]:
data.to_parquet(os.path.join(sentiment_analysis_data_path, "split_reviews.parquet"))